In [1]:
from selenium import webdriver
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import time

In [2]:
urll = 'https://onland.kbstar.com/quics?page=C060737'

In [3]:
dir_for_shinhan = 'C:/Users/shic/Desktop/shinhan_simon/chromedriver/chromedriver.exe'
dir_for_seunghun = 'C:/Users/rsh15/Desktop/seunghuni/selenium/chromedriver.exe'

In [4]:
driver = webdriver.Chrome(dir_for_shinhan)
driver.maximize_window()
driver.refresh()

In [5]:
driver.get(urll)

In [6]:
big_region_code = []
big_region_name = []

select_big_region = driver.find_element_by_id('법정동대지역코드')
options = [x for x in select_big_region.find_elements_by_tag_name('option')]
for element in options:
    big_region_code.append(element.get_attribute('value'))
    big_region_name.append(element.get_attribute('text'))
    
big_region_data = pd.DataFrame({
    'big_region_code' : big_region_code,
    'big_region_name' : big_region_name
})
big_region_data

,big_region_code,big_region_name
0,9800000000,전국
1,1100000000,서울
2,4100000000,경기
3,2800000000,인천
4,9803000000,수도권
5,2600000000,부산
6,2700000000,대구
7,2900000000,광주
8,3000000000,대전
9,3100000000,울산


In [7]:
big_region_url = 'https://onland.kbstar.com/quics?page=C060737&%EB%B2%95%EC%A0%95%EB%8F%99%EB%8C%80%EC%A7%80%EC%97%AD%EC%BD%94%EB%93%9C={}'

small_region_list = []
for big_region in big_region_data.iterrows():
    
    small_region_code = []
    small_region_name = []

    driver.get(big_region_url.format(str(big_region[1]['big_region_code'])))
    
    time.sleep(3)
    
    select_small_region = driver.find_element_by_id('법정동중지역코드')
    options = [x for x in select_small_region.find_elements_by_tag_name('option')]
    for small_element in options:
        small_region_code.append(small_element.get_attribute('value'))
        small_region_name.append(small_element.get_attribute('text'))

    small_region_tmp = pd.DataFrame({
        'big_region_code'   : big_region[1]['big_region_code'],
        'big_region_name'   : big_region[1]['big_region_name'],
        'small_region_code' : small_region_code,
        'small_region_name' : small_region_name
    })
    small_region_list.append(small_region_tmp)

driver.close()

In [18]:
kb_regions = pd.concat(small_region_list,axis=0)
kb_regions['big_region_code'] = kb_regions['big_region_code'].astype(str)
kb_regions['small_region_code'] = kb_regions['small_region_code'].astype(str)
kb_regions.loc[kb_regions['small_region_name'] == '(전체)','small_region_code'] = ' '
kb_regions

,big_region_code,big_region_name,small_region_code,small_region_name
0,9800000000,전국,,(전체)
0,1100000000,서울,,(전체)
1,1100000000,서울,1199000000,서울강북
2,1100000000,서울,1130500000,서울강북구
3,1100000000,서울,1121500000,서울광진구
...,...,...,...,...
9,4800000000,경남,4817000000,진주
10,4800000000,경남,4825000000,김해
11,4800000000,경남,4822000000,통영
0,5000000000,제주/서귀포,,(전체)


In [19]:
dirr = 'C:/Users/shic/Desktop/crawler_project/data_save/kb_land/'
kb_regions.to_csv(dirr+'region_code.csv',index=False)